# Entregable 2

## Implementación  y Población 

* IMPORTACION DE LIBRERIAS DE Python 

In [1]:
import sqlite3 
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [3]:
maxplanck=pymysql.connect(host='13.85.29.21', port=3306, user='admin', password='admin', db='maxplanck')

In [4]:
cursor=maxplanck.cursor()

* Lectura de tablas (método 1)

In [5]:
try:
    with cursor:
        sql = "select * from personas"
        cursor.execute(sql)
        cursos = cursor.fetchall()
        #print(cursos)
finally:
    pass

* Lectura de tablas (método 2)

In [6]:
personas = pd.read_sql_query("select  registros.idpers, personas.idpers, nombres, apellidos, dni, direccion from personas inner join registros on personas.idpers=registros.idpers ", maxplanck)

In [7]:
registros = pd.read_sql_query("SELECT idreg, idarea, nivel, grado, bimestre, anio as año from registros", maxplanck) #idpers,

In [8]:
areas = pd.read_sql_query("select r.idarea, a.abreviatura as Curso from registros r inner join areas a on r.idarea=a.idarea", maxplanck)

In [9]:
detregistros = pd.read_sql_query("SELECT idreg,  nota1, nota2, nota3, nota4,nota5, nota6, nota7, nota8 FROM detregistros", maxplanck)

* Creando tablas en SQLite

In [10]:
try:
        engine = create_engine('sqlite:///maxplanck.db', echo=False)
        personas.to_sql('personas', con=engine)
        registros.to_sql('registros', con=engine)
        areas.to_sql('areas', con=engine)
        detregistros.to_sql('detregistros', con=engine)
        print("Se crearon correctamente las tablas")  
except sqlite3.OperationalError:print("Las tablas ya existen") 

ValueError: Table 'personas' already exists.

## Persistencia y Serialización

In [11]:
import pickle
import pandas as pd
import pymysql
#from sqlalchemy 
import sqlalchemy
import sqlite3

In [12]:
con = sqlite3.connect('maxplanck.db')

* Lectura de tablas de maxplanck.sql

In [13]:
sql="select * from areas"
cursos=pd.read_sql_query(sql,con)
cursos.head(2)

,index,idarea,Curso
0,0,1,ALEM
1,1,1,ALEM


In [14]:
sql="select *  from personas"
alum=pd.read_sql_query(sql,con)
alum.head(2)

,index,idpers,nombres,apellidos,dni,direccion
0,0,10500,CARLOS,ARAUJO VERA,,None
1,1,10500,CARLOS,ARAUJO VERA,,None


### Serializacion

* Tabla 'Curso'

In [15]:
### funcion "dump" guardar datos
with open("cursos.csv", "bw") as archivo:
    pickle.dump(cursos, archivo)

In [16]:
### funcion "Load" cargar datos
with open("cursos.csv", "br") as archivo:
    curso=pickle.load(archivo)
    #print(reg)

* Tabla 'alumnos'

In [17]:
### funcion "dump" guardar datos
with open("alumnos.csv", "bw") as archivo:
    pickle.dump(alum, archivo)

### Persistencia

In [18]:
key ='1'
data = alum # Tabla 'ALUMNO'
#[(12345, "BASE DE DATOS"), (12346, "DAPP")]

In [19]:
import shelve

d = shelve.open('respaldo.db')  # open -- file may get suffix added by low-level
                           # library

d[key] = data              # store data at key (overwrites old data if
                           # using an existing key)
data = d[key]              # retrieve a COPY of data at key (raise KeyError
                           # if no such key)
del d[key]                 # delete data stored at key (raises KeyError
                           # if no such key)

flag = key in d            # true if the key exists
klist = list(d.keys())     # a list of all existing keys (slow!)

# as d was opened WITHOUT writeback=True, beware:
d['xx'] = [0, 1, 2]        # this works as expected, but...
d['xx'].append(3)          # *this doesn't!* -- d['xx'] is STILL [0, 1, 2]!

# having opened d without writeback=True, you need to code carefully:
temp = d['xx']             # extracts the copy
temp.append(5)             # mutates the copy
d['xx'] = temp             # stores the copy right back, to persist it

d.close()                  # close it

### Especificación del Dataset

* Conexión a la Base de Datos

In [21]:
maxplanck=pymysql.connect(host='13.85.29.21', port=3306, user='admin', password='admin', db='maxplanck')

* Lectura de tablas

In [22]:
personas = pd.read_sql_query("select  registros.idpers, personas.idpers, nombres, apellidos, dni, direccion from personas inner join registros on personas.idpers=registros.idpers ", maxplanck)

In [23]:
registros = pd.read_sql_query("SELECT idreg,  idarea, nivel, grado, bimestre, anio as año from registros", maxplanck) #idpers,

In [24]:
areas = pd.read_sql_query("select r.idarea, a.abreviatura as Curso from registros r inner join areas a on r.idarea=a.idarea", maxplanck)

In [25]:
detregistros = pd.read_sql_query("SELECT idreg,  nota1, nota2, nota3, nota4,nota5, nota6, nota7, nota8 FROM detregistros", maxplanck)

* Formación del Dataset

In [26]:
ve1=pd.concat([personas, registros], axis=1)

In [27]:
ve2=pd.merge(registros, detregistros, on='idreg')

In [28]:
ve3=pd.concat([areas, registros], axis=1)

In [29]:
maxplan=pd.concat([ve1, ve2, ve3], axis=1)

In [30]:
maxplan.head(3)

,idpers,idpers,nombres,apellidos,dni,direccion,idreg,idarea,nivel,grado,...,nota7,nota8,idarea,Curso,idreg,idarea,nivel,grado,bimestre,año
0,10500.0,10500.0,CARLOS,ARAUJO VERA,,None,1.0,22.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,1.0,22.0,PRIMARIA,1.0,1.0,2013.0
1,10500.0,10500.0,CARLOS,ARAUJO VERA,,None,2.0,12.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,2.0,12.0,PRIMARIA,1.0,1.0,2013.0
2,10500.0,10500.0,CARLOS,ARAUJO VERA,,None,3.0,26.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,3.0,26.0,PRIMARIA,1.0,1.0,2013.0


* Creación del Dataset

In [41]:
maxplan.to_csv('colegiomp.csv')

### ESTADÍSTICA DESCRIPTIVA

* Leyendo archivo .csv

In [42]:
mp=pd.read_csv('colegiomp.csv', index_col=0)
mp.head(3)

,idpers,idpers.1,nombres,apellidos,dni,direccion,idreg,idarea,nivel,grado,...,nota7,nota8,idarea.2,Curso,idreg.2,idarea.3,nivel.2,grado.2,bimestre.2,año.2
0,10500.0,10500.0,CARLOS,ARAUJO VERA,NaN,NaN,1.0,22.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,1.0,22.0,PRIMARIA,1.0,1.0,2013.0
1,10500.0,10500.0,CARLOS,ARAUJO VERA,NaN,NaN,2.0,12.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,2.0,12.0,PRIMARIA,1.0,1.0,2013.0
2,10500.0,10500.0,CARLOS,ARAUJO VERA,NaN,NaN,3.0,26.0,PRIMARIA,1.0,...,NaN,NaN,1.0,ALEM,3.0,26.0,PRIMARIA,1.0,1.0,2013.0


* MEDIA() 

In [33]:
mp.mean()

idpers        11424.148592
idpers.1      11424.148592
dni                    NaN
direccion              NaN
idreg          6417.810027
idarea           19.760965
grado             3.318741
bimestre          2.500289
año            2015.859503
idreg.1        5487.173576
idarea.1         18.467858
grado.1           3.443539
bimestre.1        2.330420
año.1          2015.446352
nota1            11.039512
nota2            12.289457
nota3            11.311630
nota4            11.329187
nota5             6.014107
nota6             9.595472
nota7             6.109439
nota8             3.731119
idarea.2         19.760965
idreg.2        6417.810027
idarea.3         19.760965
grado.2           3.318741
bimestre.2        2.500289
año.2          2015.859503
dtype: float64

* MÁXIMO()

In [34]:
mp.max()

idpers             18019
idpers.1           18019
dni                  NaN
direccion            NaN
idreg              12878
idarea                43
grado                  6
bimestre               4
año                 2018
idreg.1            12869
idarea.1              43
nivel.1       SECUNDARIA
grado.1                6
bimestre.1             4
año.1               2018
nota1                 20
nota2                 20
nota3                 20
nota4                 20
nota5                 20
nota6                 20
nota7                 20
nota8                 20
idarea.2              43
idreg.2            12878
idarea.3              43
grado.2                6
bimestre.2             4
año.2               2018
dtype: object

* MÍNIMO()

In [35]:
mp.min()

idpers           10500
idpers.1         10500
dni                NaN
direccion          NaN
idreg                1
idarea               1
grado                1
bimestre             1
año               2013
idreg.1              1
idarea.1             1
nivel.1       PRIMARIA
grado.1              1
bimestre.1           1
año.1             2013
nota1                0
nota2                0
nota3                0
nota4                0
nota5                0
nota6                0
nota7                0
nota8                0
idarea.2             1
idreg.2              1
idarea.3             1
grado.2              1
bimestre.2           1
año.2             2013
dtype: object

* CUARTILES()

In [36]:
mp.quantile()

idpers        10557.0
idpers.1      10557.0
dni               NaN
direccion         NaN
idreg          6407.0
idarea           20.0
grado             3.0
bimestre          3.0
año            2016.0
idreg.1        5627.0
idarea.1         19.0
grado.1           4.0
bimestre.1        2.0
año.1          2016.0
nota1            13.0
nota2            14.0
nota3            13.0
nota4            14.0
nota5             0.0
nota6            12.0
nota7             0.0
nota8             0.0
idarea.2         20.0
idreg.2        6407.0
idarea.3         20.0
grado.2           3.0
bimestre.2        3.0
año.2          2016.0
Name: 0.5, dtype: float64

* PERCETILES()

In [37]:
a=np.array([mp.nota1, mp.nota2, mp.nota3, mp.nota4, mp.nota5, mp.nota6, mp.nota7, mp.nota8])
p=np.percentile(a, 100)
p

/home/prueba/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


nan

### CONSULTAS SQL

In [39]:
maxplanck=pymysql.connect(host='13.85.29.21', port=3306, user='admin', password='admin', db='maxplanck')

In [40]:
#CANTIDAD DE ALUMNOS APROBADOS POR NIVEL Y AÑO
c1=pd.read_sql("""select areas.area, registros.nivel
from areas 
inner join registros 
on areas.idarea=registros.idarea
where registros.nivel='PRIMARIA' 
group by registros.nivel, areas.area
union
select areas.area, registros.nivel
from areas
inner join registros 
on areas.idarea=registros.idarea
where registros.nivel='SECUNDARIA' 
group by registros.nivel, areas.area""", maxplanck)
c1.head(3)

,area,nivel
0,ALEMÁN,PRIMARIA
1,ARTE,PRIMARIA
2,CIENCIA Y AMBIENTE,PRIMARIA


In [55]:
# CANTIDAD DE ALUMNOS APROBADOS CON PROMEDIO ALTO POR GRADO. NIVEL Y AÑO"""
c2=pd.read_sql("""select count(r.idpers) as CantAprobados14,r.grado, r.nivel, r.anio as Año from detregistros d
inner join registros r on r.idreg = d.idreg
#inner join areas a on a.idarea = r.idarea
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
#and r.nivel='PRIMARIA' / r.nivel='SECUNDARIA'
group by r.anio, r.grado, r.nivel  order by r.anio asc""", maxplanck)
c2.head(3)

,CantAprobados14,grado,nivel,Año
0,25,1,PRIMARIA,2015
1,31,1,SECUNDARIA,2015
2,44,2,PRIMARIA,2015


In [56]:
#CANTIDAD DE ALUMNOS APROBADOS CON PROMEDIO ALTO POR BIMESTRE, GRADO Y NIVEL
c3=pd.read_sql("""select distinct count(r.idpers) as CantAprobados14,r.bimestre, r.grado, r.seccion , r.nivel from detregistros d
inner join registros r on r.idreg = d.idreg
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
group by r.bimestre, r.anio, r.grado, r.nivel  
order by r.bimestre asc, r.grado asc, r.nivel asc, r.anio desc""", maxplanck)
c3.head(3)

,CantAprobados14,bimestre,grado,seccion,nivel
0,71,1,1,U,PRIMARIA
1,6,1,1,A,PRIMARIA
2,47,1,1,U,SECUNDARIA


In [57]:
#CANTIDAD DE ALUMNOS APROBADOS PROMEDIO ALTO POR BIMESTRE, GRADO, NIVEL Y AÑO
c4=pd.read_sql("""select count(r.idpers) as CantAprobados14,r.bimestre, r.grado, r.nivel, r.anio as Año from detregistros d
inner join registros r on r.idreg = d.idreg
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
group by r.bimestre, r.anio, r.grado, r.nivel  
order by r.bimestre asc, r.grado asc, r.nivel asc, r.anio desc""", maxplanck)
c4.head(3)

,CantAprobados14,bimestre,grado,nivel,Año
0,71,1,1,PRIMARIA,2018
1,6,1,1,PRIMARIA,2017
2,47,1,1,SECUNDARIA,2018


In [58]:
#CANTIDAD DE ALUMNOS APROBADOS POR CURSO, BIMESTRE , GRADO, NIVEL Y AÑO
c5=pd.read_sql("""select count(r.idpers) as CantAprobados14,r.bimestre, r.grado, r.nivel, r.anio as Año from detregistros d
inner join registros r on r.idreg = d.idreg
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
group by r.bimestre, r.anio, r.grado, r.nivel  
order by r.bimestre asc, r.grado asc, r.nivel asc, r.anio desc""", maxplanck)
c5.head(3)

,CantAprobados14,bimestre,grado,nivel,Año
0,71,1,1,PRIMARIA,2018
1,6,1,1,PRIMARIA,2017
2,47,1,1,SECUNDARIA,2018


In [59]:
#CANTIDAD DE ALUMNOS APROBADOS POR CURSO Y AÑO
c6=pd.read_sql("""select count(r.idpers) as CantiAproba11,a.area as Curso,r.anio as Año from detregistros d
inner join registros r on r.idreg = d.idreg
inner join areas a on a.idarea = r.idarea
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
#order by r.anio asc,a.area asc;
group by a.area,r.anio  order by r.anio asc""", maxplanck)
c6.head(3)

,CantiAproba11,Curso,Año
0,85,COMPUTACIÓN,2015
1,73,PERSONAL SOCIAL,2015
2,309,MATEMÁTICA,2015


In [60]:
# CANTIDAD DE ALUMNOS APROBADOS CON PROMEDIO ALTO POR CURSO Y AÑO DEL TODO EL COLEGIO"""
c7=pd.read_sql("""select count(r.idpers) as CantiAprobado14,a.area AS Cursos,r.anio  
from detregistros d
inner join registros r on r.idreg = d.idreg
inner join areas a on a.idarea = r.idarea
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
#order by r.anio asc,a.area asc;
group by a.area,r.anio  order by r.anio asc""", maxplanck)
c7.head(3)

,CantiAprobado14,Cursos,anio
0,121,ARTE,2015
1,6,"HISTORIA, GEOGRAFÍA Y ECONOMÍA",2015
2,72,COMPUTACIÓN,2015


In [61]:
# CANTIDAD DE ALUMNOS APROBADOS EN NIVEL DE PRIMARIA POR AÑO
c8=pd.read_sql("""select distinct count(r.idpers) as CantiAprobado11,r.grado,r.nivel , r.anio
from detregistros d
inner join registros r on r.idreg = d.idreg
inner join areas a on a.idarea = r.idarea
where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
and r.nivel='PRIMARIA'
group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c8.head(3)

,CantiAprobado11,grado,nivel,anio
0,85,1,PRIMARIA,2015
1,67,1,PRIMARIA,2015
2,24,1,PRIMARIA,2016


In [62]:
#CANTIDAD DE ALUMNOS APROBADOS EN NIVEL DE SECUNDARIA POR AÑO
c9=pd.read_sql("""select distinct count(r.idpers) as CantiAprobado11,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.nivel='SECUNDARIA'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c9.head(3)

,CantiAprobado11,grado,nivel,anio
0,68,1,SECUNDARIA,2015
1,108,1,SECUNDARIA,2015
2,69,1,SECUNDARIA,2015


In [63]:
#CANTIDAD DE ALUMNOS APROBADOS CON PROMEDIO ALTO EN EL CURSO DE MATEMATICA POR GRADO, NIVEL Y AÑO
c10=pd.read_sql("""select distinct count(r.idpers) as CantiAprobado14,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and a.area='MATEMATICA'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c10.head(3)

,CantiAprobado14,grado,nivel,anio
0,15,1,PRIMARIA,2015
1,34,1,SECUNDARIA,2015
2,17,1,SECUNDARIA,2017


In [64]:
#CANTIDAD DE ALUMNOS APROBADOS CON PROMEDIO ALTO EN EL CURSO DE COMUNICACION POR GRADO, NIVEL Y AÑO
c11=pd.read_sql("""select count(r.idpers) as CantiAprobado14,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=14 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and a.area='COMUNICACION'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c11.head(3)

,CantiAprobado14,grado,nivel,anio
0,9,1,PRIMARIA,2015
1,43,1,SECUNDARIA,2015
2,153,1,SECUNDARIA,2016


In [65]:
#CANTIDAD DE ALUMNOS POR GRADO, NIVEL Y AÑO
c12=pd.read_sql("""select distinct count(p.idpers), r.grado, r.nivel
    from personas p
    inner join registros r
    on r.idpers=p.idpers
    group by r.nivel, r.grado""", maxplanck)
c12.head(3)

,count(p.idpers),grado,nivel
0,808,1,PRIMARIA
1,800,2,PRIMARIA
2,800,3,PRIMARIA


In [66]:
#CANTIDAD DE ALUMNOS APROBADOS POR CURSO EN EL AÑO 2018
c13=pd.read_sql("""select count(r.idpers) as CantAprobados11,a.area AS Cursos, r.grado, r.nivel,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) >=11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.anio=2018
    group by a.area, r.nivel, r.grado  
    order by r.grado asc""", maxplanck)
c13.head(3)

,CantAprobados11,Cursos,grado,nivel,Año
0,4,ARTE,1,PRIMARIA,2018
1,93,COMUNICACIÓN,1,SECUNDARIA,2018
2,3,MATEMÁTICA,1,SECUNDARIA,2018


In [67]:
#CANTIDAD TOTAL DE ALUMNOS POR NIVEL SECUNDARIA Y PRIMARIA
c14=pd.read_sql("""select distinct count(r.idpers) as CantidadAlumnos, r.grado,r.nivel ,r.anio as Año
    from registros r
    inner join personas p
    on p.idpers=r.idpers
    where nivel='PRIMARIA' 
	group by r.anio, r.grado
    UNION
	select distinct count(r.idpers) as Cantidad, r.grado,r.nivel , r.anio as Año
    from registros r
    inner join personas p
    on p.idpers=r.idpers
    where nivel='SECUNDARIA' 
    group by r.anio, r.grado""", maxplanck)
c14.head(3)

,CantidadAlumnos,grado,nivel,Año
0,96,1,PRIMARIA,2013
1,96,2,PRIMARIA,2013
2,96,3,PRIMARIA,2013


In [68]:
#CUANTOS CURSOS LLEVAN POR NIVEL
c15=pd.read_sql("""select a.area, r.nivel
    from areas a 
    inner join registros r 
    on a.idarea=r.idarea
    where r.nivel='PRIMARIA' 
    group by r.nivel, a.area
    union
	select a.area, r.nivel
    from areas a 
    inner join registros r 
    on a.idarea=r.idarea
    where r.nivel='SECUNDARIA' 
    group by r.nivel, a.area""", maxplanck)
c15.head(3)

,area,nivel
0,ALEMÁN,PRIMARIA
1,ARTE,PRIMARIA
2,CIENCIA Y AMBIENTE,PRIMARIA


In [69]:
#CANTIDAD DE ALUMNOS DESAPROBADOS POR  NIVEL Y AÑO
c16=pd.read_sql("""SELECT count(r.idpers) as CantiDesaproba11, r.nivel, r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join personas p on p.idpers= r.idpers
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
	group by r.nivel, r.anio order by r.grado asc, r.nivel asc, r.anio asc; #group by r.grado""", maxplanck)
c16.head(3)

,CantiDesaproba11,nivel,anio
0,3516,PRIMARIA,2016
1,1449,PRIMARIA,2017
2,6404,SECUNDARIA,2015


In [70]:
#CANTIDAD DE ALUMNOS DESAPROBADOS CON PROMEDIO BAJO POR GRADO. NIVEL Y AÑO
c17=pd.read_sql("""select count(r.idpers) as CantDesaprobados14,r.grado, r.nivel, r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    #inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    #and r.nivel='PRIMARIA' / r.nivel='SECUNDARIA'
    group by r.anio, r.grado, r.nivel  order by r.anio asc""", maxplanck)
c17.head(3)

,CantDesaprobados14,grado,nivel,Año
0,192,1,PRIMARIA,2015
1,982,1,SECUNDARIA,2015
2,320,2,PRIMARIA,2015


In [71]:
#CANTIDAD DE ALUMNOS DESAPROBADOS CON PROMEDIO BAJO POR BIMESTRE, GRADO Y NIVEL
c18=pd.read_sql("""select distinct count(r.idpers) as CantDesaprobados14,r.bimestre, r.grado, r.seccion , r.nivel from detregistros d
    inner join registros r on r.idreg = d.idreg
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    group by r.bimestre, r.anio, r.grado, r.nivel  
    order by r.bimestre asc, r.grado asc, r.nivel asc, r.anio desc""", maxplanck)
c18.head(3)

,CantDesaprobados14,bimestre,grado,seccion,nivel
0,26,1,1,A,PRIMARIA
1,55,1,1,U,PRIMARIA
2,153,1,1,U,PRIMARIA


In [72]:
#CANTIDAD DE ALUMNOS DESAPROBADOS PROMEDIO BAJO POR BIMESTRE, GRADO, NIVEL Y AÑO
c19=pd.read_sql("""select count(r.idpers) as CantDesaprobados14,r.bimestre, r.grado, r.nivel, r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    group by r.bimestre, r.anio, r.grado, r.nivel  
    order by r.bimestre asc, r.grado asc, r.nivel asc, r.anio desc""", maxplanck)
c19.head(3)

,CantDesaprobados14,bimestre,grado,nivel,Año
0,26,1,1,PRIMARIA,2018
1,55,1,1,PRIMARIA,2017
2,153,1,1,PRIMARIA,2016


In [73]:
#CANTIDAD DE ALUMNOS DESAPROBADOS POR CURSO, BIMESTRE , GRADO, NIVEL Y AÑO
c20=pd.read_sql("""select count(r.idpers) as CantDesaprobados11,a.area AS Cursos, r.grado, r.nivel,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    group by a.area,r.anio  
    order by r.anio asc, r.grado asc""", maxplanck)
c20.head(3)

,CantDesaprobados11,Cursos,grado,nivel,Año
0,174,COMPUTACIÓN,1,PRIMARIA,2015
1,433,EDUCACIÓN PARA EL TRABAJO,1,SECUNDARIA,2015
2,421,FORMACIÓN. CIUDADANA Y CIVICA,1,SECUNDARIA,2015


In [74]:
#CANTIDAD DE ALUMNOS DESAPROBADOS POR CURSO Y AÑO
c21=pd.read_sql("""select count(r.idpers) as CantDesaprobados11,a.area as Curso,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    #order by r.anio asc,a.area asc;
    group by a.area,r.anio  order by r.anio asc""", maxplanck)
c21.head(3)

,CantDesaprobados11,Curso,Año
0,617,ALEMÁN,2015
1,421,FORMACIÓN. CIUDADANA Y CIVICA,2015
2,312,ARTE,2015


In [75]:
#CANTIDAD DE ALUMNOS DESAPROBADOS CON PROMEDIO BAJO POR CURSO Y AÑO DEL TODO EL COLEGIO
c22=pd.read_sql("""select count(r.idpers) as CantDesaprobados14,a.area AS Cursos,r.anio  
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    #order by r.anio asc,a.area asc;
    group by a.area,r.anio  order by r.anio asc""", maxplanck)
c22.head(3)

,CantDesaprobados14,Cursos,anio
0,617,ALEMÁN,2015
1,421,FORMACIÓN. CIUDADANA Y CIVICA,2015
2,312,ARTE,2015


In [76]:
#CANTIDAD DE ALUMNOS DESAPROBADOS EN NIVEL DE PRIMARIA POR AÑO
c23=pd.read_sql("""select distinct count(r.idpers) as CantDesaprobados11,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.nivel='PRIMARIA'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c23.head(3)

,CantDesaprobados11,grado,nivel,anio
0,314,1,PRIMARIA,2015
1,108,1,PRIMARIA,2015
2,174,1,PRIMARIA,2015


In [77]:
#CANTIDAD DE ALUMNOS DESAPROBADOS EN NIVEL DE SECUNDARIA POR AÑO
c24=pd.read_sql("""select distinct count(r.idpers) as CantDesaprobados11,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.nivel='SECUNDARIA'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c24.head(3)

,CantDesaprobados11,grado,nivel,anio
0,456,1,SECUNDARIA,2015
1,986,1,SECUNDARIA,2015
2,433,1,SECUNDARIA,2015


In [78]:
#CANTIDAD DE ALUMNOS DESAPROBADOS CON PROMEDIO BAJO EN EL CURSO DE MATEMATICA POR GRADO, NIVEL Y AÑO
c25=pd.read_sql("""select distinct count(r.idpers) as CantDesaprobados14,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and a.area='MATEMATICA'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c25.head(3)

,CantDesaprobados14,grado,nivel,anio
0,990,1,SECUNDARIA,2015
1,1895,1,SECUNDARIA,2016
2,391,1,PRIMARIA,2016


In [79]:
#CANTIDAD DE ALUMNOS DESAPROBADOS CON PROMEDIO BAJO EN EL CURSO DE COMUNICACION POR GRADO, NIVEL Y AÑO
c26=pd.read_sql("""select count(r.idpers) as CantDesaprobados14,r.grado,r.nivel , r.anio
    from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and a.area='COMUNICACION'
    group by a.area,r.anio, r.nivel order by r.grado, r.anio asc""", maxplanck)
c26.head(3)

,CantDesaprobados14,grado,nivel,anio
0,908,1,SECUNDARIA,2015
1,1599,1,SECUNDARIA,2016
2,315,1,PRIMARIA,2016


In [80]:
#CANTIDAD DE ALUMNOS POR GRADO, NIVEL Y AÑO
c27=pd.read_sql("""select distinct count(p.idpers), r.grado, r.nivel
    from personas p
    inner join registros r
    on r.idpers=p.idpers
    group by r.nivel, r.grado""", maxplanck)
c27.head(3)

,count(p.idpers),grado,nivel
0,808,1,PRIMARIA
1,800,2,PRIMARIA
2,800,3,PRIMARIA


In [81]:
#CANTIDAD DE ALUMNOS DESAPROBADOS POR CURSO EN EL AÑO 2017
c28=pd.read_sql("""select count(r.idpers) as CantDesaprobados11,a.area AS Cursos, r.grado, r.nivel,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.anio=2017
    group by a.area, r.nivel, r.grado  
    order by r.grado asc""", maxplanck)
c28.head(3)

,CantDesaprobados11,Cursos,grado,nivel,Año
0,6,ARTE,1,PRIMARIA,2017
1,27,CIENCIA Y AMBIENTE,1,PRIMARIA,2017
2,42,FORMACIÓN. CIUDADANA Y CIVICA,1,SECUNDARIA,2017


In [82]:
#CANTIDAD DE ALUMNOS DESAPROBADOS POR CURSO EN EL AÑO 2016
c29=pd.read_sql("""select count(r.idpers) as CantDesaprobados11,a.area AS Cursos, r.grado, r.nivel,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.anio=2016
    group by a.area, r.nivel, r.grado  
    order by r.grado asc""", maxplanck)
c29.head(3)

,CantDesaprobados11,Cursos,grado,nivel,Año
0,133,COMUNICACIÓN,1,SECUNDARIA,2016
1,110,EDUCACIÓN RELIGIOSA,1,SECUNDARIA,2016
2,215,MATEMÁTICA,1,SECUNDARIA,2016


In [83]:
#cANTIDAD DE ALUMNOS DESAPROBADOS POR CURSO EN EL AÑO 2015
c30=pd.read_sql("""select count(r.idpers) as CantDesaprobados11,a.area AS Cursos, r.grado, r.nivel,r.anio as Año from detregistros d
    inner join registros r on r.idreg = d.idreg
    inner join areas a on a.idarea = r.idarea
    where ((d.nota1+d.nota2+d.nota3+d.nota4+d.nota5+d.nota6+d.nota7+d.nota8)/8) <11 #and r.anio=2015#and a.area ='ARTE' and r.anio=2015
    and r.anio=2015
    group by a.area, r.nivel, r.grado  
    order by r.grado asc""", maxplanck)
c30.head(3)

,CantDesaprobados11,Cursos,grado,nivel,Año
0,22,ARTE,1,PRIMARIA,2015
1,84,ALEMÁN,1,SECUNDARIA,2015
2,22,COMPUTACIÓN,1,PRIMARIA,2015
